http://info.nec.go.kr/

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
ecode = ['3', '8','11']
ename= ['시·도지사선거',
 '광역의원비례대표선거',
 '교육감선거',]

In [ ]:
ecode_name = dict(zip(ecode, ename))
ecode_name

In [ ]:
eocode_name_sub = {"4" : '구·시·군의 장선거', 
                   "5": '시·도의회의원선거', 
                   "6" : '구·시·군의회의원선거',
                   "9" : '기초의원비례대표선거', 
                   "10" : '교육의원선거',
                   "2" : '국회의원선거'}
eocode_name_sub

In [ ]:
# ecode_name["4"]
# ecode_name["9"]

In [ ]:
def get_city_list(electionCode):
    city_code_url = "http://info.nec.go.kr/bizcommon/selectbox/selectbox_cityCodeBySgJson.json"
    city_code_param = f"electionId=0020220601&electionCode={electionCode}"

    response = requests.post(city_code_url, params=city_code_param)
    city_list = response.json()["jsonResult"]["body"]
    city_dict = {}
    for c in city_list:
        city_dict[c["CODE"]] = c["NAME"]
    return city_dict

In [ ]:
for electionCode in ecode_name:
    print(electionCode, ecode_name[electionCode])

In [ ]:
election_city_dict = { electionCode: get_city_list(electionCode) for electionCode in ecode_name}
election_city_dict

In [ ]:
election_city_dict["3"]

In [ ]:
url = "http://info.nec.go.kr/bizcommon/selectbox/selectbox_cityCodeBySgJson.json?electionId=0020220601&electionCode=3"

In [ ]:

response = requests.post(url)
response

In [ ]:
pd.DataFrame(city_list)

In [ ]:
url_xhtml = "http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml"
params = "electionId=0020220601&requestURI=%2FWEB-INF%2Fjsp%2Felectioninfo%2F0020220601%2Fcp%2Fcpri03.jsp&topMenuId=CP&secondMenuId=CPRI03&menuId=CPRI03&statementId=CPRI03_%233&electionCode=3&cityCode=1100&proportionalRepresentationCode=0&townCode=-1&dateCode=0&x=73&y=23"
url_xhtml

In [ ]:
response = requests.post(url_xhtml, params=params)

In [ ]:
html = bs(response.text)
table = html.select(".table01")[0]

In [ ]:
pd.read_html(response.text)[0]

In [ ]:
def get_city_page(electionCode, city_code):
    url_xhtml = "http://info.nec.go.kr/electioninfo/electionInfo_report.xhtml"
    params = "electionId=0020220601&requestURI=%2FWEB-INF%2Fjsp%2Felectioninfo%2F0020220601%2Fcp%2Fcpri03.jsp"
    params = params + "&topMenuId=CP&secondMenuId=CPRI03&menuId=CPRI03&statementId=CPRI03_%233"
    params = f"{params}&electionCode={electionCode}&cityCode={city_code}&proportionalRepresentationCode=0&townCode=-1&dateCode=0&x=73&y=23"

    df_temp = pd.DataFrame()
    response = requests.post(url_xhtml, params=params)
    df_temp = pd.read_html(response.text)[0]
    df_temp["선거구분"] = ecode_name[electionCode]
    df_temp["선거구"] = city_dict[city_code]
    return df_temp

In [ ]:
for eckey in ecode_name:
    print(eckey, ecode_name[eckey])

In [ ]:
city_temp_list = []
for electionCode in ecode_name:
    for city_code in city_dict:
        try:
            temp = get_city_page(electionCode, city_code)
            city_temp_list.append(temp)
        except:
            continue

In [ ]:
city_temp_list

In [ ]:
df_all = pd.concat(city_temp_list)

In [ ]:
df_all

In [ ]:
df_all.to_csv("후보자명부.csv")

In [ ]:
df_all.shape